In [15]:
import pandas as pd
import plotly.express as px

Spending

In [16]:
spending_df = pd.read_csv('data/Data_export_12042024.csv', parse_dates = ['Date'], date_format = "%d/%m/%Y")
rename_columns = {'Date' : 'date', 'Narrative' : 'narrative', 'Debit Amount' : 'debit_amount'}
spending_df = spending_df.rename(
    rename_columns, axis = 1
)[rename_columns.values()].query(
    'debit_amount >= 0'
)

In [17]:
spending_df = spending_df.assign(is_withdrawal = spending_df.narrative.str.startswith('WITHDRAWAL'))
spending_df = spending_df.query('~is_withdrawal and date >= "2024-03-01"')

In [18]:
spending_df = spending_df.groupby('date')[['debit_amount']].sum()

In [19]:
fig = px.line(spending_df)
fig.write_image('charts/spending.png')
fig

Instgram

In [20]:
actions_df = pd.read_csv('data/instagram_actions.csv', index_col = False)
actions_df = actions_df.query('date_time >= "2024-03-01"')

In [21]:
actions_df = actions_df.groupby('date').count().rename({'date_time' : 'count'}, axis = 1)

In [22]:
fig = px.line(actions_df)
fig.write_image(file = 'charts/instagram_actions.png')
fig

drugs

In [25]:
drugs_df = pd.read_csv('data/drug_consumption.csv')

In [26]:
drugs_df = pd.merge(
    drugs_df,
    drugs_df.groupby('drug')[['dose_mg']].max().rename({'dose_mg' : 'dose_max'}, axis = 1).reset_index(),
    on = 'drug'
)

drugs_df = drugs_df.assign(dose_perc = drugs_df.dose_mg / drugs_df.dose_max)

In [27]:
drugs_df = drugs_df.pivot(index = 'date_time', columns = 'drug', values = ['dose_perc'])
drugs_df.columns = drugs_df.columns.droplevel(0)

In [28]:
fig = px.line(drugs_df)
fig.write_image('charts/drug_consumption.png')
fig